***GENERATED CODE FOR classificationdataprep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def renameColumns(df, functionsData, applyOn):
    for functionData in functionsData:
        df = df.rename(
            columns={functionData['oldName']: functionData['newName']})
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '177', 'mean': '', 'stddev': '', 'min': 'ARVADA', 'max': 'WHEAT RIDGE', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Category'}, {'feature_label': 'Product_Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Product_Sub-Category', 'transformation_label': 'String Indexer'}], 'feature': 'Product_Sub-Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'Appliances', 'max': 'Telephones and Communication', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Product_Sub-Category'}, {'feature_label': 'Product_Sub-Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Product_Sub-Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'}], 'feature': 'Customer Segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Small Business', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer Segment'}, {'feature_label': 'Customer Segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer Segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'First Name', 'transformation_label': 'String Indexer'}], 'feature': 'First Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'ALXXXX', 'max': 'WX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'First Name'}, {'feature_label': 'First Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('First Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Last Name', 'transformation_label': 'String Indexer'}], 'feature': 'Last Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'ABXXXXX', 'max': 'YOXXXXXXXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Last Name'}, {'feature_label': 'Last Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Last Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Address', 'transformation_label': 'String Indexer'}], 'feature': 'Address', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': '1011 S VALENTIA ST', 'max': '9826 NEWLAND CT', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Address'}, {'feature_label': 'Address', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Address')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'CO', 'max': 'CO', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MOSAIC HOUSEHOLD', 'transformation_label': 'String Indexer'}], 'feature': 'MOSAIC HOUSEHOLD', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'A01', 'max': 'U00', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC HOUSEHOLD'}, {'feature_label': 'MOSAIC HOUSEHOLD', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MOSAIC HOUSEHOLD')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MOSAIC DESCRIPTION', 'transformation_label': 'String Indexer'}], 'feature': 'MOSAIC DESCRIPTION', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'Autumn Years', 'max': 'Young City Solos', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MOSAIC DESCRIPTION'}, {'feature_label': 'MOSAIC DESCRIPTION', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MOSAIC DESCRIPTION')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Channel', 'transformation_label': 'String Indexer'}], 'feature': 'Channel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '177', 'mean': '', 'stddev': '', 'min': 'Catalog', 'max': 'eCommerce', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Channel'}, {'feature_label': 'Channel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Channel')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run classificationdataprepHooks.ipynb
try:
	#sourcePreExecutionHook()

	customeracquisitiontrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/CustomerAcquisitionTrain.csv', 'filename': 'CustomerAcquisitionTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://13.59.224.229:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run classificationdataprepHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(customeracquisitiontrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/CustomerAcquisitionTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/CustomerAcquisitionTrain.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": [], "FilePath": "/FileStore/platform/extra/613a00515352de22a976f92a1631191476/0part.csv", "requestRatio": 7.0, "totalRows": 3275, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 24, "numberOfRows": 3275, "duplicateRowCount": 0, "stats": [{"column": "City", "alias": "City", "generated": 0, "type": "String", "max": "WHEAT RIDGE", "min": "ARVADA", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Product_Category", "alias": "Product_Category", "generated": 0, "type": "String", "max": "Technology", "min": "Furniture", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Product_Sub-Category", "alias": "Product_Sub-Category", "generated": 0, "type": "String", "max": "Telephones and Communication", "min": "Appliances", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Count", "alias": "Count", "generated": 0, "type": "numeric", "max": 4, "min": 1, "mean": 1.0778625954198473, "missing": 0.0, "stddev": 0.3}, {"column": "Customer ID", "alias": "Customer ID", "generated": 0, "type": "numeric", "max": 3402, "min": 2, "mean": 1750.4097709923665, "missing": 0.0, "stddev": 949.64}, {"column": "Store Number", "alias": "Store Number", "generated": 0, "type": "numeric", "max": 105, "min": 100, "mean": 102.34503816793894, "missing": 0.0, "stddev": 1.81}, {"column": "Customer Segment", "alias": "Customer Segment", "generated": 0, "type": "String", "max": "Small Business", "min": "Consumer", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "First Name", "alias": "First Name", "generated": 0, "type": "String", "max": "ZEXXX", "min": "AAXXX", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Last Name", "alias": "Last Name", "generated": 0, "type": "String", "max": "ZLXX", "min": "ABXXXX", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Address", "alias": "Address", "generated": 0, "type": "String", "max": "9956 CLARK DR", "min": "10 MEADOW ROSE LN", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "State", "alias": "State", "generated": 0, "type": "String", "max": "CO", "min": "CO", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Zip", "alias": "Zip", "generated": 0, "type": "numeric", "max": 80602, "min": 80002, "mean": 80114.19755725191, "missing": 0.0, "stddev": 102.36}, {"column": "DriveTime", "alias": "DriveTime", "generated": 0, "type": "real", "max": 30.0, "min": 0.65, "mean": 13.306491613740457, "missing": 0.0, "stddev": 7.38}, {"column": "Length of Residense", "alias": "Length of Residense", "generated": 0, "type": "numeric", "max": 55, "min": 0, "mean": 10.942595419847327, "missing": 0.0, "stddev": 10.76}, {"column": "MOR BANK: UPSCALE MERCHANDISE BUYER", "alias": "MOR BANK: UPSCALE MERCHANDISE BUYER", "generated": 0, "type": "numeric", "max": 4, "min": 0, "mean": 0.013129770992366412, "missing": 0.0, "stddev": 0.14}, {"column": "MOSAIC HOUSEHOLD", "alias": "MOSAIC HOUSEHOLD", "generated": 0, "type": "String", "max": "U00", "min": "A01", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "MOSAIC DESCRIPTION", "alias": "MOSAIC DESCRIPTION", "generated": 0, "type": "String", "max": "Young City Solos", "min": "Aspirational Fusion", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Customer_Lon", "alias": "Customer_Lon", "generated": 0, "type": "real", "max": -104.65312, "min": -105.226518, "mean": -104.92348986656489, "missing": 0.0, "stddev": 0.12}, {"column": "Customer_Lat", "alias": "Customer_Lat", "generated": 0, "type": "real", "max": 39.998058, "min": 39.433415999999994, "mean": 39.694812226870226, "missing": 0.0, "stddev": 0.09}, {"column": "Store ID", "alias": "Store ID", "generated": 0, "type": "numeric", "max": 105, "min": 100, "mean": 102.34503816793894, "missing": 0.0, "stddev": 1.81}, {"column": "Store_Lon", "alias": "Store_Lon", "generated": 0, "type": "real", "max": -104.717928, "min": -105.077754, "mean": -104.9181044161832, "missing": 0.0, "stddev": 0.12}, {"column": "Store_Lat", "alias": "Store_Lat", "generated": 0, "type": "real", "max": 39.856274, "min": 39.565799, "mean": 39.694891601832055, "missing": 0.0, "stddev": 0.08}, {"column": "Channel", "alias": "Channel", "generated": 0, "type": "String", "max": "eCommerce", "min": "Catalog", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Test column", "alias": "Test column", "generated": 1, "type": "numeric", "max": 4, "min": 1, "mean": 1.0778625954198473, "missing": 0.0, "stddev": 0.3}]}, "predictionPowerScore": [{"Address": 1.0, "Channel": 0.0, "City": 0.0073733468, "Count": 0.0, "Customer ID": 0.8308599894, "Customer Segment": 0.0, "Customer_Lat": 0.8272381569, "Customer_Lon": 0.8276101608, "DriveTime": 0.6760950087, "First Name": 0.2366737122, "Last Name": 0.4307625458, "Length of Residense": 0.0316601333, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0055695781, "MOSAIC HOUSEHOLD": 0.0266976309, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0446688085}, {"Address": 0.0230109615, "Channel": 1.0, "City": 0.0042471002, "Count": 0.0, "Customer ID": 0.0143099543, "Customer Segment": 0.0, "Customer_Lat": 0.0195486378, "Customer_Lon": 0.0142236187, "DriveTime": 0.0392245897, "First Name": 0.0295365291, "Last Name": 0.0382167325, "Length of Residense": 0.0012889714, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0010026315, "MOSAIC DESCRIPTION": 0.0182788807, "MOSAIC HOUSEHOLD": 0.0391384534, "Product_Category": 0.0, "Product_Sub-Category": 0.1525103346, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0231379031}, {"Address": 0.8312378291, "Channel": 0.0, "City": 1.0, "Count": 0.0, "Customer ID": 0.8057553021, "Customer Segment": 0.0, "Customer_Lat": 0.8744694103, "Customer_Lon": 0.8958548181, "DriveTime": 0.7059300079, "First Name": 0.3038633038, "Last Name": 0.4700589377, "Length of Residense": 0.1416352596, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.1519997292, "MOSAIC HOUSEHOLD": 0.2707162754, "Product_Category": 0.0, "Product_Sub-Category": 0.0331890135, "State": 0.0, "Store ID": 0.3768658705, "Store Number": 0.3768658705, "Store_Lat": 0.3768658705, "Store_Lon": 0.3768658705, "Test column": 0.0, "Zip": 1.0}, {"Address": 0.0, "Channel": 0.0, "City": 0.0, "Count": 1.0, "Customer ID": 0.0, "Customer Segment": 0.0, "Customer_Lat": 0.0, "Customer_Lon": 0.0, "DriveTime": 0.0, "First Name": 0.0, "Last Name": 0.0, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 1.0, "Zip": 0.0}, {"Address": 0.7625869503, "Channel": 0.0, "City": 0.0100853748, "Count": 0.0, "Customer ID": 1.0, "Customer Segment": 0.0008895593, "Customer_Lat": 0.6858159219, "Customer_Lon": 0.6804490055, "DriveTime": 0.5112037576, "First Name": 0.1429297991, "Last Name": 0.3268667977, "Length of Residense": 0.0177549633, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0129624675, "MOSAIC HOUSEHOLD": 0.0243991903, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0227061919}, {"Address": 0.821181115, "Channel": 0.0, "City": 0.0, "Count": 0.0, "Customer ID": 0.9363824981, "Customer Segment": 1.0, "Customer_Lat": 0.7991442288, "Customer_Lon": 0.7829514311, "DriveTime": 0.6678100966, "First Name": 0.3080406627, "Last Name": 0.4773318278, "Length of Residense": 0.0975259305, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0967048679, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.1372409841}, {"Address": 0.7930160399, "Channel": 0.0, "City": 0.5190713064, "Count": 0.0, "Customer ID": 0.7070379121, "Customer Segment": 0.0, "Customer_Lat": 1.0, "Customer_Lon": 0.7547608674, "DriveTime": 0.5227468656, "First Name": 0.1045783996, "Last Name": 0.2997879022, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0306168468, "MOSAIC HOUSEHOLD": 0.0725471895, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.4267424757, "Store Number": 0.4267424757, "Store_Lat": 0.4267424757, "Store_Lon": 0.4267424757, "Test column": 0.0, "Zip": 0.8534499765}, {"Address": 0.7224115886, "Channel": 0.0, "City": 0.5462075379, "Count": 0.0, "Customer ID": 0.7370183413, "Customer Segment": 0.0, "Customer_Lat": 0.7996574601, "Customer_Lon": 1.0, "DriveTime": 0.5885144952, "First Name": 0.1811531045, "Last Name": 0.3405745034, "Length of Residense": 0.0348419916, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0070674778, "MOSAIC HOUSEHOLD": 0.1120799028, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.5248570991, "Store Number": 0.5248570991, "Store_Lat": 0.5248570991, "Store_Lon": 0.5248570991, "Test column": 0.0, "Zip": 0.8806533181}, {"Address": 0.7816843197, "Channel": 0.0, "City": 0.1045695699, "Count": 0.0, "Customer ID": 0.7311917719, "Customer Segment": 0.0, "Customer_Lat": 0.7631344155, "Customer_Lon": 0.7564623177, "DriveTime": 1.0, "First Name": 0.191234348, "Last Name": 0.307071608, "Length of Residense": 0.0163188695, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.003835186, "MOSAIC HOUSEHOLD": 0.0248588542, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.3034811337}, {"Address": 0.8479538317, "Channel": 0.0, "City": 0.0110076174, "Count": 0.0, "Customer ID": 0.816805232, "Customer Segment": 0.0, "Customer_Lat": 0.7963909352, "Customer_Lon": 0.7963376169, "DriveTime": 0.6608809754, "First Name": 1.0, "Last Name": 0.4492261573, "Length of Residense": 0.0475243859, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.008699649, "MOSAIC HOUSEHOLD": 0.0412454238, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.072654268}, {"Address": 0.8411640948, "Channel": 0.0, "City": 0.0095063322, "Count": 0.0, "Customer ID": 0.8187574342, "Customer Segment": 0.0, "Customer_Lat": 0.7936119728, "Customer_Lon": 0.7965316262, "DriveTime": 0.6601798471, "First Name": 0.2483771873, "Last Name": 1.0, "Length of Residense": 0.0385013959, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0072764261, "MOSAIC HOUSEHOLD": 0.035547575, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0540786552}, {"Address": 0.7417743805, "Channel": 0.0, "City": 0.0, "Count": 0.0, "Customer ID": 0.7285898755, "Customer Segment": 0.0, "Customer_Lat": 0.7228430214, "Customer_Lon": 0.7251357583, "DriveTime": 0.5428067731, "First Name": 0.1777277777, "Last Name": 0.2977151606, "Length of Residense": 1.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.2235279127, "MOSAIC HOUSEHOLD": 0.3272649114, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0238553103}, {"Address": 0.581523128, "Channel": 0.0, "City": 0.0, "Count": 0.0, "Customer ID": 0.2095152649, "Customer Segment": 0.0, "Customer_Lat": 0.3489222913, "Customer_Lon": 0.5369062479, "DriveTime": 0.0574296349, "First Name": 0.0, "Last Name": 0.1564750694, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 1.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0}, {"Address": 0.8457146441, "Channel": 0.0, "City": 0.0179426473, "Count": 0.0, "Customer ID": 0.8034848543, "Customer Segment": 0.0, "Customer_Lat": 0.8202171323, "Customer_Lon": 0.8124905732, "DriveTime": 0.6629257288, "First Name": 0.3080205767, "Last Name": 0.4600843582, "Length of Residense": 0.1580503428, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 1.0, "MOSAIC HOUSEHOLD": 0.9983376885, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.001777453, "Store Number": 0.001777453, "Store_Lat": 0.001777453, "Store_Lon": 0.001777453, "Test column": 0.0, "Zip": 0.1773750157}, {"Address": 0.8604878241, "Channel": 0.0, "City": 0.0260582597, "Count": 0.0, "Customer ID": 0.8080027219, "Customer Segment": 0.0, "Customer_Lat": 0.8182231965, "Customer_Lon": 0.8125081335, "DriveTime": 0.6784103585, "First Name": 0.2841376374, "Last Name": 0.4517347178, "Length of Residense": 0.1334143981, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.3824493762, "MOSAIC HOUSEHOLD": 1.0, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0151893612, "Store Number": 0.0151893612, "Store_Lat": 0.0151893612, "Store_Lon": 0.0151893612, "Test column": 0.0, "Zip": 0.1587115495}, {"Address": 0.0, "Channel": 0.0, "City": 0.0, "Count": 0.0, "Customer ID": 0.0, "Customer Segment": 0.0, "Customer_Lat": 0.0, "Customer_Lon": 0.0, "DriveTime": 0.0, "First Name": 0.0261034558, "Last Name": 0.0, "Length of Residense": 0.0010633221, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 1.0, "Product_Sub-Category": 1.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0}, {"Address": 0.0, "Channel": 0.0, "City": 0.0, "Count": 0.0, "Customer ID": 0.0, "Customer Segment": 0.0, "Customer_Lat": 0.0, "Customer_Lon": 0.0, "DriveTime": 0.0, "First Name": 0.0, "Last Name": 0.0, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 0.0738159731, "Product_Sub-Category": 1.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0}, {"Address": 0.0, "Channel": 0.0, "City": 0.0, "Count": 0.0, "Customer ID": 0.0, "Customer Segment": 0.0, "Customer_Lat": 0.0, "Customer_Lon": 0.0, "DriveTime": 0.0, "First Name": 0.0, "Last Name": 0.0, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 1.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 0.0, "Zip": 0.0}, {"Address": 0.754589332, "Channel": 0.0, "City": 0.2713619016, "Count": 0.0, "Customer ID": 0.7605435734, "Customer Segment": 0.0, "Customer_Lat": 0.7734044578, "Customer_Lon": 0.7963807253, "DriveTime": 0.5983384017, "First Name": 0.2046305161, "Last Name": 0.347107863, "Length of Residense": 0.0350866592, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0225677416, "MOSAIC HOUSEHOLD": 0.0936770535, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 1.0, "Store Number": 1.0, "Store_Lat": 1.0, "Store_Lon": 1.0, "Test column": 0.0, "Zip": 0.3641982162}, {"Address": 0.754589332, "Channel": 0.0, "City": 0.2713619016, "Count": 0.0, "Customer ID": 0.7605435734, "Customer Segment": 0.0, "Customer_Lat": 0.7734044578, "Customer_Lon": 0.7963807253, "DriveTime": 0.5983384017, "First Name": 0.2046305161, "Last Name": 0.347107863, "Length of Residense": 0.0350866592, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0225677416, "MOSAIC HOUSEHOLD": 0.0936770535, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 1.0, "Store Number": 1.0, "Store_Lat": 1.0, "Store_Lon": 1.0, "Test column": 0.0, "Zip": 0.3641982162}, {"Address": 0.7883514233, "Channel": 0.0, "City": 0.4340318898, "Count": 0.0, "Customer ID": 0.7127088794, "Customer Segment": 0.0, "Customer_Lat": 0.8712867444, "Customer_Lon": 0.7700788551, "DriveTime": 0.5030452374, "First Name": 0.0804684419, "Last Name": 0.270854195, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 1.0, "Store Number": 1.0, "Store_Lat": 1.0, "Store_Lon": 1.0, "Test column": 0.0, "Zip": 0.5940709852}, {"Address": 0.7271961579, "Channel": 0.0, "City": 0.4128012535, "Count": 0.0, "Customer ID": 0.7396604377, "Customer Segment": 0.0, "Customer_Lat": 0.8087897846, "Customer_Lon": 0.8814783696, "DriveTime": 0.5642098796, "First Name": 0.1495794175, "Last Name": 0.3288079603, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0395785841, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 1.0, "Store Number": 1.0, "Store_Lat": 1.0, "Store_Lon": 1.0, "Test column": 0.0, "Zip": 0.6383915574}, {"Address": 0.0, "Channel": 0.0, "City": 0.0, "Count": 1.0, "Customer ID": 0.0, "Customer Segment": 0.0, "Customer_Lat": 0.0, "Customer_Lon": 0.0, "DriveTime": 0.0, "First Name": 0.0, "Last Name": 0.0, "Length of Residense": 0.0, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0, "MOSAIC DESCRIPTION": 0.0, "MOSAIC HOUSEHOLD": 0.0, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.0, "Store Number": 0.0, "Store_Lat": 0.0, "Store_Lon": 0.0, "Test column": 1.0, "Zip": 0.0}, {"Address": 0.7930007534, "Channel": 0.0, "City": 0.9474118966, "Count": 0.0, "Customer ID": 0.7640354409, "Customer Segment": 0.0, "Customer_Lat": 0.8326118881, "Customer_Lon": 0.8840909074, "DriveTime": 0.6540993248, "First Name": 0.208588419, "Last Name": 0.3617755232, "Length of Residense": 0.0618334506, "MOR BANK: UPSCALE MERCHANDISE BUYER": 0.0006998369, "MOSAIC DESCRIPTION": 0.0660056348, "MOSAIC HOUSEHOLD": 0.1933805708, "Product_Category": 0.0, "Product_Sub-Category": 0.0, "State": 0.0, "Store ID": 0.2179133626, "Store Number": 0.2179133626, "Store_Lat": 0.2179133626, "Store_Lon": 0.2179133626, "Test column": 0.0, "Zip": 1.0}], "HasBasicStats": 1, "functionsApplied": [{"functionName": "editname", "applyOn": [{"columnName": "Test column", "type": "numeric", "min": "1.0", "max": "4.0", "mean": "1.1"}], "functionsData": [{"oldName": "Count", "newName": "Test column", "asNewColumn": 1, "column": {"columnName": "Count", "type": "numeric", "min": "1.0", "max": "4.0", "mean": "1.1"}}]}], "functionChanges": [{"columnName": "Test column", "functionName": "Edit Column Name", "Type": "numeric", "Parameters": [{"oldName": "Count", "newName": "Test column", "asNewColumn": 1, "column": {"columnName": "Count", "type": "numeric", "min": "1.0", "max": "4.0", "mean": "1.1"}}]}], "fileheader": [{"field": "City", "alias": "City", "generated": 0, "position": 1, "type": "String"}, {"field": "Product_Category", "alias": "Product_Category", "generated": 0, "position": 2, "type": "String"}, {"field": "Product_Sub-Category", "alias": "Product_Sub-Category", "generated": 0, "position": 3, "type": "String"}, {"field": "Count", "alias": "Count", "generated": 0, "position": 4, "type": "numeric"}, {"field": "Customer ID", "alias": "Customer ID", "generated": 0, "position": 5, "type": "numeric"}, {"field": "Store Number", "alias": "Store Number", "generated": 0, "position": 6, "type": "numeric"}, {"field": "Customer Segment", "alias": "Customer Segment", "generated": 0, "position": 7, "type": "String"}, {"field": "First Name", "alias": "First Name", "generated": 0, "position": 8, "type": "String"}, {"field": "Last Name", "alias": "Last Name", "generated": 0, "position": 9, "type": "String"}, {"field": "Address", "alias": "Address", "generated": 0, "position": 10, "type": "String"}, {"field": "State", "alias": "State", "generated": 0, "position": 11, "type": "String"}, {"field": "Zip", "alias": "Zip", "generated": 0, "position": 12, "type": "numeric"}, {"field": "DriveTime", "alias": "DriveTime", "generated": 0, "position": 13, "type": "real"}, {"field": "Length of Residense", "alias": "Length of Residense", "generated": 0, "position": 14, "type": "numeric"}, {"field": "MOR BANK: UPSCALE MERCHANDISE BUYER", "alias": "MOR BANK: UPSCALE MERCHANDISE BUYER", "generated": 0, "position": 15, "type": "numeric"}, {"field": "MOSAIC HOUSEHOLD", "alias": "MOSAIC HOUSEHOLD", "generated": 0, "position": 16, "type": "String"}, {"field": "MOSAIC DESCRIPTION", "alias": "MOSAIC DESCRIPTION", "generated": 0, "position": 17, "type": "String"}, {"field": "Customer_Lon", "alias": "Customer_Lon", "generated": 0, "position": 18, "type": "real"}, {"field": "Customer_Lat", "alias": "Customer_Lat", "generated": 0, "position": 19, "type": "real"}, {"field": "Store ID", "alias": "Store ID", "generated": 0, "position": 20, "type": "numeric"}, {"field": "Store_Lon", "alias": "Store_Lon", "generated": 0, "position": 21, "type": "real"}, {"field": "Store_Lat", "alias": "Store_Lat", "generated": 0, "position": 22, "type": "real"}, {"field": "Channel", "alias": "Channel", "generated": 0, "position": 23, "type": "String"}, {"field": "Test column", "alias": "Test column", "generated": 1, "position": 24, "type": "numeric"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run classificationdataprepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "ARVADA", "max": "WHEAT RIDGE", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Product_Category", "transformation_label": "String Indexer"}], "feature": "Product_Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Category"}, {"transformationsData": [{"feature_label": "Product_Sub-Category", "transformation_label": "String Indexer"}], "feature": "Product_Sub-Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "Appliances", "max": "Telephones and Communication", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Product_Sub-Category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "1.11", "stddev": "0.39", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": "Count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "1748.54", "stddev": "972.83", "min": "15", "max": "3394", "missing": "0"}, "updatedLabel": "Customer ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store Number", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "102.54", "stddev": "1.94", "min": "100", "max": "105", "missing": "0"}, "updatedLabel": "Store Number"}, {"transformationsData": [{"feature_label": "Customer Segment", "transformation_label": "String Indexer"}], "feature": "Customer Segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "Consumer", "max": "Small Business", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer Segment"}, {"transformationsData": [{"feature_label": "First Name", "transformation_label": "String Indexer"}], "feature": "First Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "ALXXXX", "max": "WX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "First Name"}, {"transformationsData": [{"feature_label": "Last Name", "transformation_label": "String Indexer"}], "feature": "Last Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "ABXXXXX", "max": "YOXXXXXXXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Last Name"}, {"transformationsData": [{"feature_label": "Address", "transformation_label": "String Indexer"}], "feature": "Address", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "1011 S VALENTIA ST", "max": "9826 NEWLAND CT", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Address"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "CO", "max": "CO", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Zip", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "80117.02", "stddev": "101.84", "min": "80002", "max": "80403", "missing": "0"}, "updatedLabel": "Zip"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DriveTime", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "177", "mean": "13.14", "stddev": "7.26", "min": "1.1", "max": "30.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DriveTime"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Length of Residense", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "10.22", "stddev": "10.35", "min": "0", "max": "50", "missing": "0"}, "updatedLabel": "Length of Residense"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MOR BANK: UPSCALE MERCHANDISE BUYER", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "MOR BANK: UPSCALE MERCHAN..."}, {"transformationsData": [{"feature_label": "MOSAIC HOUSEHOLD", "transformation_label": "String Indexer"}], "feature": "MOSAIC HOUSEHOLD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "A01", "max": "U00", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC HOUSEHOLD"}, {"transformationsData": [{"feature_label": "MOSAIC DESCRIPTION", "transformation_label": "String Indexer"}], "feature": "MOSAIC DESCRIPTION", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "Autumn Years", "max": "Young City Solos", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MOSAIC DESCRIPTION"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "177", "mean": "-104.92", "stddev": "0.12", "min": "-105.226518", "max": "-104.721789", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "177", "mean": "39.7", "stddev": "0.08", "min": "39.503032", "max": "39.926565000000004", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer_Lat"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "177", "mean": "102.54", "stddev": "1.94", "min": "100", "max": "105", "missing": "0"}, "updatedLabel": "Store ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lon", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "177", "mean": "-104.92", "stddev": "0.11", "min": "-105.077754", "max": "-104.717928", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lon"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Store_Lat", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "177", "mean": "39.7", "stddev": "0.07", "min": "39.565799", "max": "39.856274", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Store_Lat"}, {"transformationsData": [{"feature_label": "Channel", "transformation_label": "String Indexer"}], "feature": "Channel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "177", "mean": "", "stddev": "", "min": "Catalog", "max": "eCommerce", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Channel"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Test column", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "177", "mean": "1.11", "stddev": "0.39", "min": "1.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Test column"}, {"feature": "City_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "1.36", "stddev": "1.88", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "City_stringindexer_transf..."}, {"feature": "Product_Category_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "177", "mean": "0.66", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Product_Category_stringin..."}, {"feature": "Product_Sub-Category_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "4.54", "stddev": "4.02", "min": "0.0", "max": "16.0", "missing": "0"}, "updatedLabel": "Product_Sub-Category_stri..."}, {"feature": "Customer Segment_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "177", "mean": "1.2", "stddev": "1.09", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Customer Segment_stringin..."}, {"feature": "First Name_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "38.95", "stddev": "32.21", "min": "0.0", "max": "108.0", "missing": "0"}, "updatedLabel": "First Name_stringindexer_..."}, {"feature": "Last Name_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "54.53", "stddev": "42.08", "min": "0.0", "max": "135.0", "missing": "0"}, "updatedLabel": "Last Name_stringindexer_t..."}, {"feature": "Address_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "72.4", "stddev": "49.18", "min": "0.0", "max": "159.0", "missing": "0"}, "updatedLabel": "Address_stringindexer_tra..."}, {"feature": "State_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "177", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "State_stringindexer_trans..."}, {"feature": "MOSAIC HOUSEHOLD_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "13.8", "stddev": "11.0", "min": "0.0", "max": "43.0", "missing": "0"}, "updatedLabel": "MOSAIC HOUSEHOLD_stringin..."}, {"feature": "MOSAIC DESCRIPTION_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "177", "mean": "5.68", "stddev": "4.53", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "MOSAIC DESCRIPTION_string..."}, {"feature": "Channel_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "177", "mean": "0.55", "stddev": "0.7", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Channel_stringindexer_tra..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run classificationdataprepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["City_stringindexer", "Product_Category_stringindexer", "Product_Sub-Category_stringindexer", "Count", "Customer ID", "Store Number", "Customer Segment_stringindexer", "First Name_stringindexer", "Last Name_stringindexer", "Address_stringindexer", "State_stringindexer", "Zip", "DriveTime", "Length of Residense", "MOR BANK: UPSCALE MERCHANDISE BUYER", "MOSAIC HOUSEHOLD_stringindexer", "MOSAIC DESCRIPTION_stringindexer", "Customer_Lon", "Customer_Lat", "Store ID", "Store_Lon", "Store_Lat", "Test column"], "Channel_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

